In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read dataset
matches = pd.read_csv("matches.csv")
deli = pd.read_csv("deliveries.csv")

# Question 1:

In [ ]:
# all matches between KKR and RCB
kkr_rcb = matches.query("team1.isin(['Kolkata Knight Riders', 'Royal Challengers Bangalore']) and team2.isin(['Kolkata Knight Riders', 'Royal Challengers Bangalore'])")

In [ ]:
# victories of the two teams against each other 
kkr_rcb.winner.value_counts()

In [ ]:
# matches of KKR
kkr = matches.query("team1 == 'Kolkata Knight Riders' or team2 == 'Kolkata Knight Riders'")
len(kkr)

In [ ]:
# count of winners in KKR matches
kkr.winner.value_counts()

In [ ]:
# KKR win ratio
92/178

In [ ]:
# matches of RCB
rcb = matches.query("team1 == 'Royal Challengers Bangalore' or team2 == 'Royal Challengers Bangalore'")
len(rcb)

In [ ]:
# count of winners in RCB matches
rcb.winner.value_counts()

In [ ]:
# RCB win ratio
84/180

# Explanation:

Based on historical data alone, it seems that KKR will be the winner of today's match.

# Question 2:

In [ ]:
# all deliveries faced by RCB in the 6th-20th overs
rcb_bat = deli[deli.batting_team == "Royal Challengers Bangalore"].query("over.isin([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])")

In [ ]:
# wickets lost by RCB in the 6th-20th overs 
wickets = rcb_bat.groupby("match_id", as_index=False).count()[["match_id", "player_dismissed"]]

In [ ]:
# histogram
plt.hist(wickets["player_dismissed"], bins=[0, 2, 4, 5, 10])
plt.title("Wickets lost by RCB in the 6th-20th overs")
plt.xlabel("No. of wickets")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# renaming column
matches = matches.rename(columns={"id":"match_id"})

In [ ]:
# merging the wickets dataframe with the matches dataframe
merge_data = wickets.merge(matches, on="match_id")

# histogram
plt.hist(merge_data[merge_data.season.isin([2018, 2019])]["player_dismissed"], bins=[0, 2, 4, 5, 10])
plt.title("Wickets lost by RCB in the 6th-20th overs in the last two seasons")
plt.xlabel("No. of wickets")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# wickets lost by RCB in the 16th-20th overs ag. KKR
wickets_kkr = merge_data.query("team1 == 'Kolkata Knight Riders' or team2 == 'Kolkata Knight Riders'")

# histogram
plt.hist(wickets_kkr["player_dismissed"], bins=[0, 2, 4, 5, 10])
plt.title("Wickets lost by RCB ag. KKR in the 6th-20th overs")
plt.xlabel("No. of wickets")
plt.ylabel("Frequency")
plt.show()

# Explanation:

Considering the previous years' data, RCB tend to lose 5 or wickets in the 6th to 20th overs, and this patterned has been followed in the last two seasons and against KKR as well. Hence RCB can be expected to lose 5 or more wickets in the 6th to 20th overs today.

# Question 3:

In [ ]:
# deliveries faced by AB de Villiers
abd = deli[deli.batsman == "AB de Villiers"]

In [ ]:
# no. of matches played by him in IPL
len(abd.match_id.unique())

In [ ]:
# total runs made by him in each of the matches
abd_30 = abd.groupby("match_id", as_index=False).sum()[["match_id", "batsman_runs"]]

In [ ]:
# no. of matches in which he did not cross the score of 30
len(abd_30[abd_30.batsman_runs <= 30])

In [ ]:
# % of matches in which he did not cross the score of 30
86/142

AB de Villiers does not cross the score of 30 in 60% of the matches.

In [ ]:
# merging the deliveries with the matches dataframe and filtering for KKR
abd_30.merge(matches, on="match_id").query("team1 == 'Kolkata Knight Riders' or team2 == 'Kolkata Knight Riders'")

Against KKR as well, he has not been able to cross the score of 30 majority of the times.

In [ ]:
# filtering out the matches in which he crossed the score of 30
abd_deli = deli[deli.match_id.isin(abd_30[abd_30.batsman_runs >= 30].match_id)]

In [ ]:
# function to calculate balls taken to score 30 runs
def get_balls_for_30(df):
    
    # empty list to store the number of balls
    num_balls = []

    # iterating over all the matches in the dataframe
    for m_id in df.match_id.unique():

        # getting the record of one match using the match id
        data = df[df.match_id == m_id]
        
        # calculating cumulative sum of the score
        cumsum = np.cumsum(data["total_runs"])
        
        # finding the index position of the ball on which the score of 100 is crossed
        end_pos = pd.DataFrame(cumsum >= 30).idxmax()[0]

        # dataframe which stores the details of the balls below the score of 100
        balls_taken_df = data.loc[:end_pos][:]

        # total balls taken
        total_balls = len(balls_taken_df)
        # extra balls like wide, no ball, etc. 
        extra_balls = len(balls_taken_df[balls_taken_df.extra_runs != 0])

        # final number of balls
        final_balls = total_balls - extra_balls
        num_balls.append(final_balls)
        
        # removing the cases where the score of 100 was not crossed
        final = [val for val in num_balls if val>1]
        
    return final

In [ ]:
# histogram
plt.hist(get_balls_for_30(abd_deli), bins=[0, 18, 24, max(get_balls_for_30(abd_deli))])
plt.title("No. of balls taken to cross the score of 30 in IPL")
plt.xlabel("No. of balls")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# No. of balls taken to cross the score of 30 in 2019
get_balls_for_30(abd_deli[abd_deli.match_id > 11000])

In [ ]:
# no. of times he has been dismissed by KKR bowlers
abd.query("bowling_team == 'Kolkata Knight Riders' & player_dismissed.notnull()").bowler.value_counts()

Explanation:

AB de Villiers does not seem to cross the score of 30 in today's match.

# Question 4:

In [ ]:
# all deliveries faced by Eoin Morgan
morgan = deli[deli.batsman == "EJG Morgan"]

In [ ]:
# balls per inning faced by him
balls_per_inning = morgan.groupby("match_id", as_index=False).count()[["match_id", "ball"]]

In [ ]:
# matches in which he has played more than 7 balls
more_than_7 = balls_per_inning[balls_per_inning.ball > 7]

In [ ]:
# dataframe of deliveries in the matches in which he has played more than 7 balls
morgan_df = morgan[morgan.match_id.isin(more_than_7.match_id)]

In [ ]:
# runs scored per match
runs = pd.DataFrame(morgan_df.groupby("match_id", as_index=False).sum()[["match_id", "batsman_runs"]])

In [ ]:
# total balls faced per match
ball = pd.DataFrame(morgan_df.groupby("match_id", as_index=False).count()[["match_id", "ball"]])

In [ ]:
# extra balls
extra = pd.DataFrame(morgan_df[morgan_df.extra_runs != 0].groupby("match_id", as_index=False).count()[["match_id", "over"]])

In [ ]:
# valid balls faced per match
balls = ball.set_index('match_id').join(extra.set_index('match_id')).fillna(0)

balls['final'] = balls["ball"] - balls["over"]

In [ ]:
balls = balls.reset_index()

In [ ]:
# creating dataframe to store strike rate
sr = runs.merge(balls, on="match_id")

In [ ]:
# calculating strike rate
sr["strike_rate"] = sr["batsman_runs"]/sr["final"]*100

sr.head()

In [ ]:
plt.hist(sr["strike_rate"])
plt.title("Strike Rate of Eoin Morgan in IPL matches")
plt.xlabel("Strike Rate")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# median strike rate
sr["strike_rate"].median()

In [ ]:
# calculating strike rate in the last 7 balls
sr_7 = []             # list to store strike rate

# iterating over the matches
for m_id in morgan_df.match_id.unique():
    
    # dataframe for each match
    d_ = morgan_df[morgan_df.match_id == m_id]
    # last 7 balls excluding extras
    data = d_[d_.extra_runs == 0].iloc[-7:]
    
    # runs on the last 7 balls
    runs = data.batsman_runs.sum()
    # no. of balls
    balls = len(data)
    
    # appending the value
    sr_7.append(runs/balls*100)

In [ ]:
# median strike rate
np.median(sr_7)

In [ ]:
# difference in strike rates
diff = np.abs(sr["strike_rate"] - sr_7)

# histogram
plt.hist(diff, bins=[0, 75, 125, 150, max(diff)])
plt.title("Difference in the strike rates")
plt.xlabel("Difference")
plt.ylabel("Frequency")
plt.show()

# Explanation:

The strike rate for Eoin Morgan will stay below 75 in today's match.

# Question 5:

In [ ]:
# all deliveries faced by RCB
rcb = deli[deli.batting_team == "Royal Challengers Bangalore"]

In [ ]:
# deliveries faced by RCB in the 16th-20th overs
rcb_16_20 = rcb.query("over.isin([16, 17, 18, 19, 20])")

In [ ]:
# runcs scored in those overs
runs = rcb_16_20.groupby("match_id").sum()["total_runs"]

In [ ]:
# histogram
plt.hist(runs, bins=[0, 35, 45, 60, max(runs)])
plt.title("Runs scored by RCB in the 16th-20th overs in IPL")
plt.xlabel("No. of runs")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# runs scored in those overs against KKR
runs_kkr = rcb_16_20.query("bowling_team == 'Kolkata Knight Riders'").groupby("match_id").sum()["total_runs"]

In [ ]:
# histogram
plt.hist(runs_kkr, bins=[0, 35, 45, 60, max(runs)])
plt.title("Runs scored by RCB in the 16th-20th overs against KKR")
plt.xlabel("No. of runs")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# runs scored in those overs in 2019
runs_2019 = rcb_16_20.query("match_id > 11000").groupby("match_id").sum()["total_runs"]

In [ ]:
# histogram
plt.hist(runs_2019, bins=[0, 35, 45, 60, max(runs)])
plt.title("Runs scored by RCB in the 16th-20th overs in 2019")
plt.xlabel("No. of runs")
plt.ylabel("Frequency")
plt.show()

# Explanation:

RCB will score 46-60 runs in the death overs in today's match.